# Import Dependencies

In [1]:
import os 
import numpy as np 
import cv2 as cv
import matplotlib.pyplot as plt

from pygments.formatters import img
from tqdm import tqdm

from utils.edge_extraction import *
from utils.feature_extraction import * 
from utils.fourier_transform import * 
from utils.image_conversion import * 
from utils.clustering import *
from utils.contrast_enhancement import *

# Pre-processing

To reduce noise in images of whole artworks and fragments, we initially considered using the Fourier transform to process the images in the frequency domain.

While converting an image from RGBA to grayscale simplifies processing, it results in the loss of RGB color and alpha channel data, which can be problematic if that information is needed later. Therefore, we chose to split the image into its primary color channels (excluding the alpha channel) and process each channel separately in the frequency domain. After filtering, we planned to reconstruct the filtered image by recombining the processed channels.

However, after several trials, we found that processing the channels separately led to significant information loss in one or more channels. Consequently, we decided to use the NLMeansDenoising filter instead.

Since our goal is to cluster fragments that belong to the same image, we focus on maintaining "continuity" along the fragment borders. Therefore, our process emphasizes the information present along these edges.

Steps:
1. Extract a working region from the borders of the fragment.
2. Filter out the transparent pixels from the working region.
3. Denoise the working region.

**CONSIDERATION**: Contrast enhancement.

In [2]:
input_dir = "./data"
files = os.listdir(input_dir)
ext = ".png"
threshold = 5

images = []
for filename in tqdm(files, total=len(files), desc="Pre-processing files: extracting working region, denoising..."): 
    if not filename.endswith(ext):
        continue 
    
    image = cv.imread(os.path.join(input_dir, filename), cv.IMREAD_UNCHANGED)
    working_region = extract_working_region(input_img=image, threshold=threshold)
    b, g, r, a = cv.split(working_region)
    working_region = filter_working_region(working_region)
    # denoise working region 
    denoised_working_region = cv.fastNlMeansDenoisingColored(working_region)
    images.append(denoised_working_region)

Pre-processing files: extracting working region, denoising...: 100%|██████████| 328/328 [00:03<00:00, 94.37it/s] 


# Feature Extraction

To extract relevant features from the fragments, we employ two methods:
- Color Histograms
- Gradient Jacobians

## Color Histograms

Color histograms are graphical representations of the distribution of colors in an image. They quantify the number of pixels that have specific color values, effectively capturing the color composition of the image. By analyzing the color histograms of image fragments, we can compare and cluster similar fragments based on their color distributions.

**This technique is particularly useful for identifying and matching regions of images that share similar color patterns**.

In [3]:
flatten_color_histograms = compute_color_histograms(images)
unflatten_color_histograms = compute_color_histograms(images, flatten=False)

Computing color histograms: 100%|██████████| 328/328 [00:00<00:00, 113527.95it/s]


In [11]:
distance_matrix_color_histogram = compute_color_histogram_dist_matrix(unflatten_color_histograms)
distance_matrix_color_histogram

Calculating similarities: 100%|██████████| 328/328 [00:00<00:00, 1331.42it/s]


array([[ 0.        ,  0.94099119,  0.70407303, ...,  0.68840727,
        18.71498171, 10.45640334],
       [ 0.94099119,  0.        ,  0.53312135, ...,  0.52093016,
         0.54443817,  2.4476498 ],
       [ 0.70407303,  0.53312135,  0.        , ...,  0.15362315,
         0.74818506,  1.14708026],
       ...,
       [ 0.68840727,  0.52093016,  0.15362315, ...,  0.        ,
         0.46151521,  8.52518236],
       [18.71498171,  0.54443817,  0.74818506, ...,  0.46151521,
         0.        ,  8.74787502],
       [10.45640334,  2.4476498 ,  1.14708026, ...,  8.52518236,
         8.74787502,  0.        ]])

### K-Means

In [5]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
fit_kmeans = kmeans.fit(flatten_color_histograms)
create_cluster_dirs(data_dir="./data", output_dir="clusters/kmeans/colors", labels=fit_kmeans.labels_)

Creating cluster dirs: 100%|██████████| 328/328 [00:00<00:00, 2700.26it/s]


### DBSCAN

In [6]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=10, metric="precomputed")
fit_db = db.fit(distance_matrix_color_histogram)
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(fit_db.labels_)) - (1 if -1 in fit_db.labels_ else 0)
n_noise_ = list(fit_db.labels_).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

create_cluster_dirs(data_dir="./data", output_dir="clusters/dbscan/colors", labels= fit_db.labels_)

Estimated number of clusters: 3
Estimated number of noise points: 140


Creating cluster dirs: 100%|██████████| 328/328 [00:00<00:00, 5360.70it/s]


## Gradient Jacobians

Gradient Jacobians represent the gradients of pixel intensities in an image. They capture the rate of change of pixel values in both the horizontal and vertical directions, highlighting edges and texture details. By computing the Jacobians of image fragments, we can compare and group fragments that exhibit similar edge and texture patterns. Formally, the gradient jacobians we use are of the form:

$$
\begin{align}
\begin{bmatrix} G_x & G_{x\_gray} \\ G_y & G_{y\_gray} \end{bmatrix}
\end{align}
$$

where $G_x$ and $G_y$ are the aggregated gradient of the RGB channels, while $G_{x\_gray}$ and $G_{y\_gray}$ are the gradient of the grayscale image.

This method is especially valuable for identifying structural similarities and continuities between different fragments.

In [7]:
flatten_jacobians = compute_jacobians(images)
unflatten_jacobians = compute_jacobians(images, flatten=False)

Computing Jacobians: 100%|██████████| 328/328 [00:00<00:00, 5636.63it/s]


In [12]:
distance_matrix_jacobians = compute_jacobians_dist_matrix(unflatten_jacobians)
distance_matrix_jacobians

Calculating similarities: 100%|██████████| 328/328 [00:24<00:00, 13.16it/s]


array([[   0.        ,  836.70160916,  958.64829566, ..., 1123.52381831,
         806.03275264,  819.01347532],
       [ 836.70160916,    0.        ,  693.29550262, ...,  902.30211815,
         396.64420592,  372.46770642],
       [ 958.64829566,  693.29550262,    0.        , ...,  993.26215825,
         654.43766903,  660.86244321],
       ...,
       [1123.52381831,  902.30211815,  993.26215825, ...,    0.        ,
         873.30937189,  882.57807076],
       [ 806.03275264,  396.64420592,  654.43766903, ...,  873.30937189,
           0.        ,  338.71324583],
       [ 819.01347532,  372.46770642,  660.86244321, ...,  882.57807076,
         338.71324583,    0.        ]])

### K-Means

In [9]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
fit_kmeans = kmeans.fit(flatten_jacobians)
create_cluster_dirs(data_dir="./data", output_dir="clusters/kmeans/jacobians", labels=fit_kmeans.labels_)

Creating cluster dirs: 100%|██████████| 328/328 [00:00<00:00, 2059.29it/s]


### DBSCAN

In [10]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=10, metric="precomputed")
fit_db = db.fit(distance_matrix_jacobians)
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(fit_db.labels_)) - (1 if -1 in fit_db.labels_ else 0)
n_noise_ = list(fit_db.labels_).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

create_cluster_dirs(data_dir="./data", output_dir="clusters/dbscan/jacobians", labels= fit_db.labels_)

Estimated number of clusters: 0
Estimated number of noise points: 328


Creating cluster dirs: 100%|██████████| 328/328 [00:00<00:00, 5326.22it/s]
